In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc, precision_recall_curve

!pip install transformers torch pandas scikit-learn

In [2]:
df = pd.read_json("train.json")
df.head(5)

# df["description"]

df["description"][4]

'Spacious 1 Bedroom 1 Bathroom in Williamsburg!Apartment Features:- Renovated Eat in Kitchen With Dishwasher- Renovated Bathroom- Beautiful Hardwood Floors- Lots of Sunlight- Great Closet Space- Freshly Painted- Heat and Hot Water Included- Live in Super Nearby L, J, M & G Trains !<br /><br />Contact Information:Kenneth BeakExclusive AgentC: 064-692-8838Email: kagglemanager@renthop.com, Text or Email to schedule a private viewing!<br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><p><a  website_redacted '

We can see here that there are html tags in the text which is not required

In [3]:
#Removal of outlier
upper_bound = np.percentile(df["price"].values, 99.9)
df_filtered = df[df["price"] <= upper_bound]

In [4]:
df_filtered["date"] = pd.to_datetime(df_filtered["created"]).dt.date
df_filtered["year"] = pd.to_datetime(df_filtered["date"]).dt.year
df_filtered["month"] = pd.to_datetime(df_filtered["date"]).dt.month
df_filtered["day"] = pd.to_datetime(df_filtered["date"]).dt.day
df_filtered["hour"] = pd.to_datetime(df_filtered["date"]).dt.hour
df_filtered["weekday"] = pd.to_datetime(df_filtered["date"]).dt.weekday
df_filtered["quarter"] = pd.to_datetime(df_filtered["date"]).dt.quarter
df_filtered.head(5)

<ipython-input-4-200f75a3fb14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["date"] = pd.to_datetime(df_filtered["created"]).dt.date
<ipython-input-4-200f75a3fb14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["year"] = pd.to_datetime(df_filtered["date"]).dt.year
<ipython-input-4-200f75a3fb14>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,...,price,street_address,interest_level,date,year,month,day,hour,weekday,quarter
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,...,2400,145 Borinquen Place,medium,2016-06-16,2016,6,16,0,3,2
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,...,3800,230 East 44th,low,2016-06-01,2016,6,1,0,2,2
9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,...,3495,405 East 56th Street,medium,2016-06-14,2016,6,14,0,1,2
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,...,3000,792 Metropolitan Avenue,medium,2016-06-24,2016,6,24,0,4,2
15,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,...,2795,340 East 34th Street,low,2016-06-28,2016,6,28,0,1,2


PREPROCESSING
- Removal of html tags
- Convert text to lowercase


In [5]:
def remove_html_tags(description):
    description = re.sub(r'<[^>]+>', '', description)
    description = re.sub(r'\s+', ' ', description)
    return description.strip()


df_filtered['processed_description'] = df_filtered['description'].apply(remove_html_tags)

df_filtered["processed_description"] = df_filtered["description"].apply(lambda desc: desc.lower())
df_filtered["processed_description_distilBERT"] = df_filtered["processed_description"]

<ipython-input-5-450b8d759a7d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['processed_description'] = df_filtered['description'].apply(remove_html_tags)
<ipython-input-5-450b8d759a7d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["processed_description"] = df_filtered["description"].apply(lambda desc: desc.lower())
<ipython-input-5-450b8d759a7d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

Tokenization : Split the text into words (tokens) for easier processing.

In [6]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
# Load spaCy's pre-trained model (small model for efficiency)
nlp = spacy.load("en_core_web_sm")

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def spaCy_tokenize(description):
    doc = nlp(description)
    return doc # Return the spaCy Doc object instead of just token texts

def remove_stopwords_and_lemmatize(doc): # Accept spaCy Doc object
    processed_words = []
    for token in doc:
        if token.text not in stop_words and not token.is_punct: # Access token.text for stop word check
            processed_words.append(token.lemma_)
    return " ".join(processed_words)
# To be removed after testing.
# df_filtered = df_filtered.iloc[:100]
df_filtered['processed_description'] = df_filtered['processed_description'].apply(spaCy_tokenize) # Store Doc objects
df_filtered['processed_description'] = df_filtered['processed_description'].apply(remove_stopwords_and_lemmatize)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-7-028db781f529>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['processed_description'] = df_filtered['processed_description'].apply(spaCy_tokenize) # Store Doc objects
<ipython-input-7-028db781f529>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['processed_description'] = df_filtered['processed_description'].apply(remove_stopwords_and_lemmatize)

In [8]:
print(df_filtered['processed_description'].head())

4     spacious 1 bedroom 1 bathroom williamsburg!apa...
6     brand new gut renovate true 2 bedroomfind home...
9     flex 2 bedroom full pressurized wall**looke pe...
10    brand new 3 bedroom 1.5 bath apartmentenjoy fo...
15    sized studio w abundant closet available immed...
Name: processed_description, dtype: object


RNN MODEL

In [9]:
# Convert to sequences

tokenizer_rnn = Tokenizer(num_words=10000)
tokenizer_rnn.fit_on_texts(df_filtered['processed_description'])
sequences_rnn = tokenizer_rnn.texts_to_sequences(df_filtered['processed_description'])

max_sequence_length_run = 100
X = pad_sequences(sequences_rnn, maxlen=max_sequence_length_run, padding='post')

In [10]:
# Initialize Model
model_rnn = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length_run),
    LSTM(128, return_sequences=True),
    Dropout(0.5),
    LSTM(64),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  #Since we want binary outputs
])

model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_rnn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Predict sentiment scores and add to dataframe that is df_filtered
sentiment_scores_rnn = model_rnn.predict(X)

print(sentiment_scores_rnn)

sentiment_labels_rnn = (sentiment_scores_rnn > 0.5).astype(int)  # 1 for positive, 0 for negative

df_filtered['sentiment_score_rnn'] = sentiment_labels_rnn

1542/1542 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step
[[0.50160795]
 [0.5027259 ]
 [0.49994272]
 ...
 [0.5016448 ]
 [0.5026281 ]
 [0.5014789 ]]


<ipython-input-11-3e2d647bbccb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['sentiment_score_rnn'] = sentiment_labels_rnn


In [12]:
print(df_filtered["sentiment_score_rnn"].describe())

df_filtered["sentiment_score_rnn"].value_counts()


count    49317.000000
mean         0.891985
std          0.310403
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: sentiment_score_rnn, dtype: float64


,count
sentiment_score_rnn,
1,43990
0,5327


**Using DistilBERT for Sentiment Analysis**

In [13]:
# Load the tokenizer and model with GPU support
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Define the batch size
BATCH_SIZE = 32  # You can adjust this based on your Colab session’s GPU memory

def preprocess_batch(texts):
    return tokenizer(texts, return_tensors="pt", max_length=512, truncation=True, padding=True)

def predict_sentiment_batch(texts):
    inputs = preprocess_batch(texts)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_classes = torch.argmax(logits, dim=1).cpu().numpy()

    # Map the numeric predictions to "positive" or "negative"
    sentiments = [1 if pred == 1 else 0 for pred in predicted_classes]
    return sentiments

# Apply sentiment analysis in batches to the 'processed_description' column
sentiments = []
batch_texts = []

for i, text in enumerate(df_filtered["processed_description_distilBERT"]):
    batch_texts.append(text)
    # Process the batch when it's full or at the end of the dataset
    if len(batch_texts) == BATCH_SIZE or i == len(df_filtered) - 1:
        sentiments.extend(predict_sentiment_batch(batch_texts))
        batch_texts = []  # Reset for the next batch

# Add the sentiments to the DataFrame
df_filtered['sentiment_distilBERT'] = sentiments

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-13-4505bb9bd65d>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['sentiment_distilBERT'] = sentiments


Evaluating RNN and DistilBert

In [14]:
def analyze_model_quality(rnn_predictions, bert_predictions):
    rnn_pred = np.array(rnn_predictions)
    bert_pred = np.array(bert_predictions)

    # Calculate metrics
    rnn_balance = abs((rnn_pred == 1).mean() - 0.5)  # How close to 50-50 split -> The model which has some kind of balance is a better model
    bert_balance = abs((bert_pred == 1).mean() - 0.5)

    print("\nModel Analysis:")
    print("==============")
    print(f"RNN Positive Predictions: {(rnn_pred == 1).mean()*100:.2f}%")
    print(f"BERT Positive Predictions: {(bert_pred == 1).mean()*100:.2f}%")
    print(f"RNN Balance Score (lower is better): {rnn_balance:.3f}")
    print(f"BERT Balance Score (lower is better): {bert_balance:.3f}")

    # Sample disagreements
    disagreements = np.where(rnn_pred != bert_pred)[0]
    if len(disagreements) > 0:
        print(f"\nTotal disagreements: {len(disagreements)} ({len(disagreements)/len(rnn_pred)*100:.2f}%)")

    # Make recommendation
    if bert_balance < rnn_balance:
        recommendation = "BERT"
        reason = "more balanced predictions"
    else:
        recommendation = "RNN"
        reason = "more balanced predictions"

    print(f"\nRecommendation: {recommendation} model appears better due to {reason}")
    return recommendation

# Get recommendation
better_model = analyze_model_quality(
    df_filtered['sentiment_score_rnn'],
    df_filtered['sentiment_distilBERT']
)




Model Analysis:
RNN Positive Predictions: 89.20%
BERT Positive Predictions: 99.99%
RNN Balance Score (lower is better): 0.392
BERT Balance Score (lower is better): 0.500

Total disagreements: 5331 (10.81%)

Recommendation: RNN model appears better due to more balanced predictions


In [15]:
# Generate final CSV with both predictions and recommended model's prediction
df_final = df_filtered.copy()
if better_model == "BERT":
    df_final['final_sentiment'] = df_final['sentiment_distilBERT']
else:
    df_final['final_sentiment'] = df_final['sentiment_score_rnn']

# Save to CSV
output_filename = 'sentiment_analysis_results.csv'
df_final.to_csv(output_filename)
#and to json
df_final.to_json("sentimental_extraction.json")

print(f"\nResults saved to {output_filename}")
print("\nColumns in final CSV:")
print(df_final.columns.tolist())
print("\nSample of final predictions:")


Results saved to sentiment_analysis_results.csv

Columns in final CSV:
['bathrooms', 'bedrooms', 'building_id', 'created', 'description', 'display_address', 'features', 'latitude', 'listing_id', 'longitude', 'manager_id', 'photos', 'price', 'street_address', 'interest_level', 'date', 'year', 'month', 'day', 'hour', 'weekday', 'quarter', 'processed_description', 'processed_description_distilBERT', 'sentiment_score_rnn', 'sentiment_distilBERT', 'final_sentiment']

Sample of final predictions:


In [18]:
df_json_output = pd.read_json('sentimental_extraction.json')

In [19]:
print(df_json_output.head())

    bathrooms  bedrooms                       building_id  \
4         1.0         1  8579a0b0d54db803821a35a4a615e97a   
6         1.0         2  b8e75fc949a6cd8225b455648a951712   
9         1.0         2  cd759a988b8f23924b5a2058d5ab2b49   
10        1.5         3  53a5b119ba8f7b61d4e010512e0dfc85   
15        1.0         0  bfb9405149bfff42a92980b594c28234   

                created                                        description  \
4   2016-06-16 05:55:27  Spacious 1 Bedroom 1 Bathroom in Williamsburg!...   
6   2016-06-01 05:44:33  BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...   
9   2016-06-14 15:19:59  **FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...   
10  2016-06-24 07:54:24  A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...   
15  2016-06-28 03:50:23  Over-sized Studio w abundant closets. Availabl...   

        display_address                                           features  \
4   145 Borinquen Place  [Dining Room, Pre-War, Laundry in Building, Di...   
6       